In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
detector = HandDetector(detectionCon=0.8)
startDistance = None
scale = 0
cx, cy = 500, 500
newH, newW = 250, 250

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType = False)
    
    if len(hands) == 1:
        if detector.fingersUp(hands[0]) == [1, 1, 1, 0, 0]:
            l, _, _ = detector.findDistance(hands[0]['lmList'][8][0:2], hands[0]['lmList'][12][0:2], img)
#         print(l)
            if l < 50:
                cursor = hands[0]['lmList'][8][0:2]
                if cx - newW//2 < cursor[0] < cx + newW//2 and cy - newH//2 < cursor[1] < cy + newH//2:
                    cx, cy = cursor
    
    if len(hands) == 2:
        if detector.fingersUp(hands[0]) == [0, 1, 0, 0, 0] and detector.fingersUp(hands[1]) == [0, 1, 0, 0, 0]:
            lmList1 = hands[0]["lmList"]
            lmList2 = hands[1]["lmList"]
            length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img)
            if startDistance is None:
                startDistance = length
            else:
                scale = (int)((length - startDistance) // 2)
                cx, cy = info[4:6]
    else:
        startDistance = None
    
    img1 = cv2.imread("cat.jpg")
    h1, w1, _ =  img1.shape
    newH, newW = (h1 + scale)//2*2, (w1 + scale)//2*2
    img1 = cv2.resize(img1, (newW, newH))
    try:
        img[cy - newH//2:cy + newH//2, cx - newW//2:cx + newW//2] = img1
    except:
        pass
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)